# 0628 1차 수행평가 2

## handwrite classify

- 숫자 필기체 이미지를 텍스트 형태로 변환 (숫자 부분은 1, 나머지는 0 으로)
- 알고리즘 2종 이상 사용, 각 알고리즘의 결과를 출력해야함
- 각 알고리즘에 아래와 같이 3가지가 출력되야함

    1. 분류기가 예측한 숫자와 실제 숫자가 맞는지 출력 (테스트 데이터 수 만큼)
    2. 에러 카운트
    3. 에러 비율
    
    
- 라이브러리 사용 자제바람
- 꼭 써야하면 위에 3가지 출력 준수
- 훈련, 테스트 데이터 합쳐서 사용해도 무방

#####  풀이는 이렇게 해보면 좋을 것 같다..

0,1 데이터로 되어있는 데이터들을 배열에 담고

    - 배열은 1차원 배열로 가도 될듯
    - 실제 필기체가 아니라서 필압의 차이가 없어서, 필압이 쎈데를 1차 배열, 필압이 약한데를 2차 배열... 이렇게 안놔도 될듯
    
전처리

    - 여기서 알고리즘 1종을 써야하는데.. 분류를 해야하니까
    - 1,2,3,....,9 까지 데이터가 들어왔을 때, 이게 1일 확률, 2일 확률, 3일 확률을 각각 산출해줘야하고
    - 모든 확률의 합은 1
    - 소프트맥스 함수 이용(해보자)
    
행렬은 몇개?

    - 숫자 데이터 행렬 x에 대해, 가중치 w와 편향 b가 붙는데
    - 각 숫자 txt 파일 까서 보면, 32글자씩 33줄이다. 32x33 = 1056
    - 일단 w는 1056개의 글자에 다 붙어야한다. 그래서 w 배열은 w.shape = [1056, 9]
    - b는 마지막에 붙으니까 걍 9
    
kNN

    - 학습을 시키고 가장 유사한 k개의 갯수를 가져오고
    - 나머지 분류가 제대로 안된게 error rate 로 되게
    
    
일단 써놓고 되는거로 해보자    

In [1]:
import numpy as np

In [9]:
# 일단 하나 불러서 어떻게 나오는지

ex = np.loadtxt("C:/Users/Affinity/Desktop/개인자료/module3/digits/trainingDigits/0_0.txt",
               delimiter=",")
ex

array([1.11100000e+18, 1.11111100e+19, 1.11111111e+21, 1.11111111e+24,
       1.11111101e+24, 1.11111100e+25, 1.11111100e+25, 1.11111100e+25,
       1.11111100e+25, 1.11111100e+25, 1.11111000e+25, 1.11111000e+25,
       1.11111000e+25, 1.11111000e+24, 1.11111000e+24, 1.11111000e+24,
       1.11111000e+24, 1.11111100e+25, 1.11101100e+25, 1.11100000e+25,
       1.11100000e+24, 1.11100000e+24, 1.11100000e+24, 1.11100000e+24,
       1.11000000e+23, 1.11100011e+23, 1.11111111e+23, 1.11111111e+22,
       1.11111111e+22, 1.11111111e+21, 1.11110000e+20, 1.10000000e+19])

In [18]:
# 위에서 float 형으로 나와서... dtype = object 로 하겠음..
# dtype = int.64로 부르면 overflow 발생, dtype=false 도 안됨.

ex = np.loadtxt("C:/Users/Affinity/Desktop/개인자료/module3/digits/trainingDigits/0_0.txt",
               delimiter=",", dtype=object)
# type(ex)
ex

# 각 숫자 하나하나가 아니라, line 한 줄이 객체 하나가 된다.

array(['00000000000001111000000000000000',
       '00000000000011111110000000000000',
       '00000000001111111111000000000000',
       '00000001111111111111100000000000',
       '00000001111111011111100000000000',
       '00000011111110000011110000000000',
       '00000011111110000000111000000000',
       '00000011111110000000111100000000',
       '00000011111110000000011100000000',
       '00000011111110000000011100000000',
       '00000011111100000000011110000000',
       '00000011111100000000001110000000',
       '00000011111100000000001110000000',
       '00000001111110000000000111000000',
       '00000001111110000000000111000000',
       '00000001111110000000000111000000',
       '00000001111110000000000111000000',
       '00000011111110000000001111000000',
       '00000011110110000000001111000000',
       '00000011110000000000011110000000',
       '00000001111000000000001111000000',
       '00000001111000000000011111000000',
       '00000001111000000000111110000000',
       '000

In [141]:
import os # 파일 한꺼번에 다부르려고 넣고
import time # Wall Time 출력 (파일이 많아서..)
import operator # kNN에서 index에 따라서 전체 데이터를 정렬하기 위해, 연산자 모듈 (미리) 넣고

from numpy import *
from numpy import array

In [30]:
%%time
# trainSet 자료를 전부 불러놓고,
for root, dirs, files in os.walk("C:/Users/Affinity/Desktop/개인자료/module3/digits/trainingDigits") :
    for frame in files :
        full_trainSet = os.path.join(root, frame)
        
#         print(full_trainSet)
# 이렇게 하면 파일에 접근을 못하는것 같아서, 그냥 path 자체를 변수로 놓고,
# os.listdir 로 부르겠음...

Wall time: 8.93 ms


In [121]:
train_path = "C:/Users/Affinity/Desktop/개인자료/module3/digits/trainingDigits"
train_isrealnum = [] # 빈 리스트를 만들고, 여기에 진짜 숫자 데이터를 넣을 예정
test_path = "C:/Users/Affinity/Desktop/개인자료/module3/digits/testDigits"
test_isrealnum = [] # 마찬가지

In [122]:
# 라인 하나하나당 객체 하나니까, 이걸 1056개의 문자로 하나하나 쪼개려면...?
def making_matrix(txtfile) :
    txt_dataArray = np.zeros((1, 1056))
    return txt_dataArray
making_matrix(ex)
# 대충 이렇게 하면 될거같은데

array([[0., 0., 0., ..., 0., 0., 0.]])

In [58]:
# enumerate for loop 돌리면 인덱스가 생긴다.
# for loop 돌릴때.... 위에서 line 하나가 객체로 불러와지니까, 각 라인마다 index가 필요하다
# for in range 말고 for in enumerate 로 해서 인덱스를 각각 붙이겠음 (index는 그럼 파일당 33개)
names = ['철수', '영희', '영수']
for i, name in enumerate(names):
      print('{}번: {}'.format(i + 1, name))

1번: 철수
2번: 영희
3번: 영수


In [128]:
# txt 파일을 행렬로 변환하는 함수
def making_matrix_index(txtfile) :
    txt_dataArray = np.zeros((1, 1056))
    # 영행렬 행은 일단 하나, 모든 데이터를 저장해야하니 1056개로 스타트
    # 이제 파일을 불러와야하는데, 
    with open(txtfile) as lines : # with ~ as 문을 쓰면 파일 닫기를 자동으로 마무리 해준다고함
        for line_num, line in enumerate(lines) : # line_num은 txt 파일안의 라인 index가 될거고, line은 그 한줄 한덩어리가
            for i in range(32) : # 각 라인에 담긴 글자가 32개
                txt_dataArray[0, 32 * line_num + i] = int(line[i])
    return txt_dataArray

In [124]:
making_matrix_index(ex)
# 바로 호출하면 안될것 같으니, 두개로 쪼개자.

C:\ProgramData\Anaconda3\envs\ywpython\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: path should be string, bytes, or os.PathLike, not numpy.ndarray
  


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xed in position 3: invalid continuation byte

##### trainingSet, testingSet 을 어떻게 쪼개면 좋을것 같냐면

    - train/test 각 세트마다, 기능을 2개로 놓고 해보기
        - 파일을 불러서 라인 > 글자 하나하나 인덱스 매기는 함수 하나
        - 폴더의 파일 죄다 부르고, 글자 하나하나 찢어서 하나로 합치는 함수 하나
    - 일단 지금은 train_set 만...    

In [131]:
def get_train_matrix() :
    # listdir 써서 모든 디렉토리의 파일 부르고,
    # 리스트 길이만큼 빈 행렬 만들어두고
    # for loop > enumerate로
    # 구분자를 붙여서 다 쪼개고 행렬에 저장 (여기 안에서 making_matrix(txtfile) 을 돌려준다.)
    
    full_trainSet = os.listdir(train_path)
    trainMatrix = np.zeros((len(full_trainSet), 1056))
    
    # file_num 이 각 파일의 index가 될거고, train_file 이 각 0_0.txt 파일들
    for file_num, train_file in enumerate(full_trainSet) :
        train_isrealnum.append(train_file.split(',')[0]) # 아까 만들어둔 실데이터 저장할 리스트에 저장
        trainMatrix[file_num, :] = making_matrix(train_path + '/' + train_file)
#         print(trainMatrix)
    return full_trainSet

In [130]:
%%time
# 함수 테스트
get_train_matrix()

# 출력결과 엄청 긴데.. print로 찍어보면 각 txt 파일이 , 로 구분되어 출력되긴됨
# 그냥 full_trainSet 부르면 txt 파일명이 리스트로 저장되어있음
# Wall time: 1.72 s

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ...

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ...

 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ..

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ...

 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Wall time: 745 ms


['0_0.txt',
 '0_1.txt',
 '0_10.txt',
 '0_100.txt',
 '0_101.txt',
 '0_102.txt',
 '0_103.txt',
 '0_104.txt',
 '0_105.txt',
 '0_106.txt',
 '0_107.txt',
 '0_108.txt',
 '0_109.txt',
 '0_11.txt',
 '0_110.txt',
 '0_111.txt',
 '0_112.txt',
 '0_113.txt',
 '0_114.txt',
 '0_115.txt',
 '0_116.txt',
 '0_117.txt',
 '0_118.txt',
 '0_119.txt',
 '0_12.txt',
 '0_120.txt',
 '0_121.txt',
 '0_122.txt',
 '0_123.txt',
 '0_124.txt',
 '0_125.txt',
 '0_126.txt',
 '0_127.txt',
 '0_128.txt',
 '0_129.txt',
 '0_13.txt',
 '0_130.txt',
 '0_131.txt',
 '0_132.txt',
 '0_133.txt',
 '0_134.txt',
 '0_135.txt',
 '0_136.txt',
 '0_137.txt',
 '0_138.txt',
 '0_139.txt',
 '0_14.txt',
 '0_140.txt',
 '0_141.txt',
 '0_142.txt',
 '0_143.txt',
 '0_144.txt',
 '0_145.txt',
 '0_146.txt',
 '0_147.txt',
 '0_148.txt',
 '0_149.txt',
 '0_15.txt',
 '0_150.txt',
 '0_151.txt',
 '0_152.txt',
 '0_153.txt',
 '0_154.txt',
 '0_155.txt',
 '0_156.txt',
 '0_157.txt',
 '0_158.txt',
 '0_159.txt',
 '0_16.txt',
 '0_160.txt',
 '0_161.txt',
 '0_162.txt',
 '0

In [168]:
def get_test_matrix2() :
    # listdir 써서 모든 디렉토리의 파일 부르고,
    # 리스트 길이만큼 빈 행렬 만들어두고
    # for loop > enumerate로
    # 구분자를 붙여서 다 쪼개고 행렬에 저장 (여기 안에서 making_matrix(txtfile) 을 돌려준다.)
    
    full_testSet = os.listdir(test_path)
    testMatrix = np.zeros((len(full_testSet), 1056))
    
    # file_num 이 각 파일의 index가 될거고, train_file 이 각 0_0.txt 파일들
    for file_num, test_file in enumerate(full_testSet) :
        test_isrealnum.append(test_file.split(',')[0]) # 아까 만들어둔 실데이터 저장할 리스트에 저장
        testMatrix[file_num, :] = making_matrix_index(test_path + '/' + test_file)
#         print(testMatrix)
    return full_testSet

In [135]:
get_test_matrix2()

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ...

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ...

['0_0.txt',
 '0_1.txt',
 '0_10.txt',
 '0_11.txt',
 '0_12.txt',
 '0_13.txt',
 '0_14.txt',
 '0_15.txt',
 '0_16.txt',
 '0_17.txt',
 '0_18.txt',
 '0_19.txt',
 '0_2.txt',
 '0_20.txt',
 '0_21.txt',
 '0_22.txt',
 '0_23.txt',
 '0_24.txt',
 '0_25.txt',
 '0_26.txt',
 '0_27.txt',
 '0_28.txt',
 '0_29.txt',
 '0_3.txt',
 '0_30.txt',
 '0_31.txt',
 '0_32.txt',
 '0_33.txt',
 '0_34.txt',
 '0_35.txt',
 '0_36.txt',
 '0_37.txt',
 '0_38.txt',
 '0_39.txt',
 '0_4.txt',
 '0_40.txt',
 '0_41.txt',
 '0_42.txt',
 '0_43.txt',
 '0_44.txt',
 '0_45.txt',
 '0_46.txt',
 '0_47.txt',
 '0_48.txt',
 '0_49.txt',
 '0_5.txt',
 '0_50.txt',
 '0_51.txt',
 '0_52.txt',
 '0_53.txt',
 '0_54.txt',
 '0_55.txt',
 '0_56.txt',
 '0_57.txt',
 '0_58.txt',
 '0_59.txt',
 '0_6.txt',
 '0_60.txt',
 '0_61.txt',
 '0_62.txt',
 '0_63.txt',
 '0_64.txt',
 '0_65.txt',
 '0_66.txt',
 '0_67.txt',
 '0_68.txt',
 '0_69.txt',
 '0_7.txt',
 '0_70.txt',
 '0_71.txt',
 '0_72.txt',
 '0_73.txt',
 '0_74.txt',
 '0_75.txt',
 '0_76.txt',
 '0_77.txt',
 '0_78.txt',
 '0_79.

In [182]:
# 같은 방식으로 test 용 함수도 만들고

def get_test_matrix() :
    full_testSet = os.listdir(test_path)
#     testMatrix = np.zeros((len(full_testSet), 1056))
    testMatrix_list = []
    
    # file_num 이 각 파일의 index가 될거고, test_file 이 각 0_0.txt 파일들
    for test_file in enumerate(full_testSet) :
        test_isrealnum.append(test_file.split(',')[0]) # 아까 만들어둔 실데이터 저장할 리스트에 저장
        # 테스트 파일들을 리스트 안에 넣음
        testMatrix_list.append(making_matrix_index(test_path + '/' + test_file))
    return testMatrix_list

In [183]:
get_test_matrix()

# 이거 말고 위에 get_test_matrix2 쓰자.

AttributeError: 'tuple' object has no attribute 'split'

##### kNN 으로 먼저 풀어보기

    - k=3 으로 지정
    - 분류기 함수 하나, 실행하는 함수 하나
    
분류기에는
    
    - 테스트용을 train만큼 복사하고,
        - tile 쓸 때, tile(테스트, (트레인, 1)) 로 해서
        - [ [테스트, 트레인], [테스트, 트레인] , ... 가 트레인txt 갯수만큼 ] 되게..
            
    - 위에 복사한 각각의 리스트에서 유클리드 거리 구하고
    - argsort() 로 거리를 오름차순 정렬
        
    - 아까 숫자들 저장해둔 array에 for문을 돌면서, label 값을 매김
    
실행기에는

    - 실제로 분류된 거랑 추측한거랑 맞으면 correct, 틀리면 error

In [110]:
# operator 모듈 잘 모르겠어서 예제 한번

category = [[200130, 32000, "Google", 5.96, 71325],
[200130, 32000, "Netflix", 5.96, 30000],
[200230, 17400, "FaceBook", 29.85, 25751012],
[206640, 18750, "Amazon", 6.53, 214676]]

print('------------Sort by 3rd index-----------------')
category.sort(key=operator.itemgetter(3))  # or newlist = sorted(category, key=itemgetter(3))

for temp in category:
    print(temp)
    
print('------------Reverse-----------------')
category.sort(key=operator.itemgetter(3), reverse=True)

for temp in category:
    print(temp)
    
print('------------Sort by multiple index-----------------')
category.sort(key=operator.itemgetter(3))

for temp in category:
    print(temp)

------------Sort by 3rd index-----------------
[200130, 32000, 'Google', 5.96, 71325]
[200130, 32000, 'Netflix', 5.96, 30000]
[206640, 18750, 'Amazon', 6.53, 214676]
[200230, 17400, 'FaceBook', 29.85, 25751012]
------------Reverse-----------------
[200230, 17400, 'FaceBook', 29.85, 25751012]
[206640, 18750, 'Amazon', 6.53, 214676]
[200130, 32000, 'Google', 5.96, 71325]
[200130, 32000, 'Netflix', 5.96, 30000]
------------Sort by multiple index-----------------
[200130, 32000, 'Google', 5.96, 71325]
[200130, 32000, 'Netflix', 5.96, 30000]
[206640, 18750, 'Amazon', 6.53, 214676]
[200230, 17400, 'FaceBook', 29.85, 25751012]


In [187]:
# kNN 분류기
# 교재에는 라벨을 매개인자로 썼는데, 아까 위에 realnum 으로 저장해둔거 있어서 그걸 그대로 쓰겠음

def knn_classify(trainMatrix, testMatrix, k=3) :
    trainMatrix = np.array(trainMatrix)
    # 행렬에서 첫줄만 뽑고
    trainMatrix_lines = trainMatrix.shape[0]
    # 테스트-트레인 하나씩 짝지어서 리스트로
    tile_testMatrix = np.tile(testMatrix, (trainMatrix_lines, 1))
    # 유클리드 거리 구하기
    sqTileMat = (tile_testMatrix - trainMatrix)** 2
    sqDistances = sqTileMat.sum(axis=1)
    E_Distances = sqDistances**0.5
    # 거리 오름차순 정렬
    sortedDistIndicies = E_Distances.argsort()
    
    classCount = {}
    # for loop
    for i in range(k) : # k=3 이니까 3번 돌아감
        # 아까 trainset labeling 해둔 데이터에 trainData[i]에 해당하는 수를 구함
        voteLabel = train_isrealnum[sortedDistIndicies[i]]
        print(voteLabel)
        # 실제 구한 수에 +1을 하는게 아니라, 발생 횟수에 +1을 해준다.
        classCount[voteLabel] = classCount.get(voteLabel, 0) + 1
    # key 값을 classcount의     
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
    
    return sortedClassCount

In [186]:
train_example = get_train_matrix()
test_example = get_test_matrix2()
knn_classify(train_example, test_example, 3)

# 그냥 테스트는 안될것 같고... 실행기를 짜고나서 다시 해야할듯

TypeError: ufunc 'subtract' did not contain a loop with signature matching types dtype('<U9') dtype('<U9') dtype('<U9')

In [139]:
# 실행기

def knn_Test() :
    trainMatrix = get_train_matrix()
    testMatrix = get_test_matrix2()
    correct = 0.0
    total = 0.0
    error = total - correct
    
    for indX, test_matrix in enumerate(testMatrix) :
        real = test_isrealnum[indX]
        guess = knn_classify(trainMatrix, test_matrix, 3)
        # 예측값과 실제값이 같으면 정답, 아니면 오답
        if real == guess :
            correct += 1
        total += 1
        print("The classifier came back with : {}, the real answer is: {}".format(guess, real))
    print("The total error rate is : {}".format(error / total))
    print("The error count : {}".format(error))

In [169]:
knn_Test()

TypeError: ufunc 'subtract' did not contain a loop with signature matching types dtype('<U9') dtype('<U9') dtype('<U9')

##### 나름대로 다르게 해보려고 하는데 에러 잘 모르겠어서, 그냥 책에 있는대로 진행하면

In [188]:
def classify0(inX, dataSet, labels, k):
    dataSetSize = dataSet.shape[0]
    diffMat = tile(inX, (dataSetSize, 1)) - dataSet
    sqDiffMat = diffMat ** 2
    sqDistances = sqDiffMat.sum(axis = 1)
    distances = sqDistances ** 0.5
    sortedDistIndicies = distances.argsort()
    classCount={}
    for i in range(k):
        voteIlabel = labels[sortedDistIndicies[i]]
        classCount[voteIlabel] = classCount.get(voteIlabel, 0) + 1
    sortedClassCount = sorted(classCount.items(),
                              key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]

In [190]:
def img2vector(filename):
    returnVect = zeros((1, 1056))
    fr = open(filename)
    for i in range(32):
        lineStr = fr.readline()
        for j in range(32):
            returnVect[0, 32*i+j] = int(lineStr[j])
    return returnVect

In [193]:
# k=3 일때의 kNN

def hwClassifier() :  
    trainLabels = []
    trainingFileList = os.listdir('C:/Users/Affinity/Desktop/개인자료/module3/digits/trainingDigits')
    m = len(trainingFileList)
    trainingMat = zeros((m, 1056))
    for i in range(m):
        fileNameStr = trainingFileList[i]
        fileStr = fileNameStr.split('.')[0]
        classNumStr = int(fileStr.split('_')[0])
        trainLabels.append(classNumStr)
        trainingMat[i, :] = img2vector('C:/Users/Affinity/Desktop/개인자료/module3/digits/trainingDigits/%s' % fileNameStr)
        
    testLabels = []
    testFileList = os.listdir('C:/Users/Affinity/Desktop/개인자료/module3/digits/testDigits')
    n = len(testFileList)
    testMat = zeros((n, 1056))
    for i in range(n):
        fileNameStr = testFileList[i]
        fileStr = fileNameStr.split('.')[0]
        classNumStr = int(fileStr.split('_')[0])
        testLabels.append(classNumStr)
        testMat[i, :] = img2vector('C:/Users/Affinity/Desktop/개인자료/module3/digits/testDigits/%s' % fileNameStr)   
        
    errorcount=0.0
    for i in range(n):
        classifierResult = classify0(testMat[i,:], trainingMat, trainLabels, 3)
        print("the classifier came back with: {}, the real answer: {}".format(classifierResult, testLabels[i]))
        if (classifierResult != testLabels[i]) : errorcount+=1.0
    print("the total number of errors is : {}".format(errorcount))
    print("the total error rate is : {}".format(errorcount/float(n)))

In [195]:
%%time
hwClassifier()

"""
the total number of errors is : 10.0
the total error rate is : 0.010570824524312896
Wall time: 18.5 s
"""

the classifier came back with: 0, the real answer: 0
the classifier came back with: 0, the real answer: 0
the classifier came back with: 0, the real answer: 0
the classifier came back with: 0, the real answer: 0
the classifier came back with: 0, the real answer: 0
the classifier came back with: 0, the real answer: 0
the classifier came back with: 0, the real answer: 0
the classifier came back with: 0, the real answer: 0
the classifier came back with: 0, the real answer: 0
the classifier came back with: 0, the real answer: 0
the classifier came back with: 0, the real answer: 0
the classifier came back with: 0, the real answer: 0
the classifier came back with: 0, the real answer: 0
the classifier came back with: 0, the real answer: 0
the classifier came back with: 0, the real answer: 0
the classifier came back with: 0, the real answer: 0
the classifier came back with: 0, the real answer: 0
the classifier came back with: 0, the real answer: 0
the classifier came back with: 0, the real ans

the classifier came back with: 1, the real answer: 1
the classifier came back with: 1, the real answer: 1
the classifier came back with: 1, the real answer: 1
the classifier came back with: 1, the real answer: 1
the classifier came back with: 1, the real answer: 1
the classifier came back with: 1, the real answer: 1
the classifier came back with: 1, the real answer: 1
the classifier came back with: 1, the real answer: 1
the classifier came back with: 7, the real answer: 1
the classifier came back with: 1, the real answer: 1
the classifier came back with: 1, the real answer: 1
the classifier came back with: 1, the real answer: 1
the classifier came back with: 1, the real answer: 1
the classifier came back with: 1, the real answer: 1
the classifier came back with: 1, the real answer: 1
the classifier came back with: 1, the real answer: 1
the classifier came back with: 1, the real answer: 1
the classifier came back with: 1, the real answer: 1
the classifier came back with: 1, the real ans

the classifier came back with: 3, the real answer: 3
the classifier came back with: 3, the real answer: 3
the classifier came back with: 3, the real answer: 3
the classifier came back with: 3, the real answer: 3
the classifier came back with: 3, the real answer: 3
the classifier came back with: 3, the real answer: 3
the classifier came back with: 3, the real answer: 3
the classifier came back with: 3, the real answer: 3
the classifier came back with: 3, the real answer: 3
the classifier came back with: 3, the real answer: 3
the classifier came back with: 3, the real answer: 3
the classifier came back with: 3, the real answer: 3
the classifier came back with: 3, the real answer: 3
the classifier came back with: 3, the real answer: 3
the classifier came back with: 3, the real answer: 3
the classifier came back with: 3, the real answer: 3
the classifier came back with: 3, the real answer: 3
the classifier came back with: 3, the real answer: 3
the classifier came back with: 3, the real ans

the classifier came back with: 5, the real answer: 5
the classifier came back with: 5, the real answer: 5
the classifier came back with: 5, the real answer: 5
the classifier came back with: 5, the real answer: 5
the classifier came back with: 5, the real answer: 5
the classifier came back with: 5, the real answer: 5
the classifier came back with: 5, the real answer: 5
the classifier came back with: 5, the real answer: 5
the classifier came back with: 5, the real answer: 5
the classifier came back with: 5, the real answer: 5
the classifier came back with: 5, the real answer: 5
the classifier came back with: 5, the real answer: 5
the classifier came back with: 5, the real answer: 5
the classifier came back with: 5, the real answer: 5
the classifier came back with: 5, the real answer: 5
the classifier came back with: 5, the real answer: 5
the classifier came back with: 5, the real answer: 5
the classifier came back with: 5, the real answer: 5
the classifier came back with: 5, the real ans

the classifier came back with: 6, the real answer: 6
the classifier came back with: 6, the real answer: 6
the classifier came back with: 6, the real answer: 6
the classifier came back with: 6, the real answer: 6
the classifier came back with: 6, the real answer: 6
the classifier came back with: 6, the real answer: 6
the classifier came back with: 6, the real answer: 6
the classifier came back with: 6, the real answer: 6
the classifier came back with: 6, the real answer: 6
the classifier came back with: 6, the real answer: 6
the classifier came back with: 6, the real answer: 6
the classifier came back with: 6, the real answer: 6
the classifier came back with: 6, the real answer: 6
the classifier came back with: 6, the real answer: 6
the classifier came back with: 6, the real answer: 6
the classifier came back with: 6, the real answer: 6
the classifier came back with: 6, the real answer: 6
the classifier came back with: 6, the real answer: 6
the classifier came back with: 6, the real ans

the classifier came back with: 8, the real answer: 8
the classifier came back with: 8, the real answer: 8
the classifier came back with: 8, the real answer: 8
the classifier came back with: 8, the real answer: 8
the classifier came back with: 8, the real answer: 8
the classifier came back with: 8, the real answer: 8
the classifier came back with: 1, the real answer: 8
the classifier came back with: 8, the real answer: 8
the classifier came back with: 8, the real answer: 8
the classifier came back with: 8, the real answer: 8
the classifier came back with: 8, the real answer: 8
the classifier came back with: 8, the real answer: 8
the classifier came back with: 8, the real answer: 8
the classifier came back with: 8, the real answer: 8
the classifier came back with: 8, the real answer: 8
the classifier came back with: 8, the real answer: 8
the classifier came back with: 1, the real answer: 8
the classifier came back with: 8, the real answer: 8
the classifier came back with: 8, the real ans

the classifier came back with: 9, the real answer: 9
the classifier came back with: 9, the real answer: 9
the total number of errors is : 10.0
the total error rate is : 0.010570824524312896
Wall time: 18.5 s


##### 소프트맥스 함수도 이용

    - 참고 URL : http://aidev.co.kr/deeplearning/841
    - https://tensorflow.blog/%ED%95%B4%EC%BB%A4%EC%97%90%EA%B2%8C-%EC%A0%84%ED%95%B4%EB%93%A4%EC%9D%80-%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D-3/

In [203]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

digits = load_digits()
digits_data = digits.data / 16
X_train, X_test, y_train, y_test = train_test_split(digits_data, digits.target,
                                   test_size=0.1)

In [212]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(n_values=10)
y_train_enc = ohe.fit_transform(y_train.reshape(-1, 1)).toarray()
print(y_train[:3])
print(y_train_enc[:3])

[5 4 1]
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]


C:\ProgramData\Anaconda3\envs\ywpython\lib\site-packages\sklearn\preprocessing\_encoders.py:331: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)]'.
  warnings.warn(msg, DeprecationWarning)


    - 참고 링크의 예제 코드에서도 이미지 전처리하면 똑같은 형태의 벡터로 나옴
    - 아까 우리가 변환했던 img2Vec 그대로 쓰겠음

소프트맥스를 쓰려면..

    - hidden layer 를 1024개보다 더 많이 두고 (txt 파일 하나당 글자갯수)
    - hidden layer 에 쓰일 활성화 함수(시그모이드) 
    - 시그모이드를 거쳐서, 소프트맥스 함수로
    - 하나의 클래스 안에 시그모이드 / 그다음 소프트맥스 / 예측 3개의 메인 함수를 깐다

In [217]:
# 위에서 썼던 trainingMat 재탕

trainLabels = []
trainingFileList = os.listdir('C:/Users/Affinity/Desktop/개인자료/module3/digits/trainingDigits')
m = len(trainingFileList)
trainingMat = zeros((m, 1056))
for i in range(m):
    fileNameStr = trainingFileList[i]
    fileStr = fileNameStr.split('.')[0]
    classNumStr = int(fileStr.split('_')[0])
    trainLabels.append(classNumStr)
    trainingMat[i, :] = img2vector('C:/Users/Affinity/Desktop/개인자료/module3/digits/trainingDigits/%s' % fileNameStr)

In [218]:
from scipy.special import expit # 시그모이드 쓰려고

In [219]:
class FCNeuron(object):

    def _sigmoid(self, y_hat):
        return expit(y_hat)

    def forpass(self, x): # y = wx + b 계산해주는거
        self._x = x
        self._t = self._sigmoid(np.dot(self._x, self._w1) + self._b1)
        _y_hat = np.dot(self._t, self._w2) + self._b2
        return self._softmax(_y_hat)
    
    def _softmax(self, y_hat):
        tmp = y_hat - y_hat.max(axis=1).reshape(-1, 1)
        exp_tmp = np.exp(tmp)
        return exp_tmp / exp_tmp.sum(axis=1).reshape(-1, 1)